In [1]:
import re

This wonderful thread fixed 2+ hours of mucking around....

http://stackoverflow.com/questions/375427/non-blocking-read-on-a-subprocess-pipe-in-python

In [44]:
import sys
from subprocess import PIPE, Popen
from threading  import Thread

try:
    from Queue import Queue, Empty
except ImportError:
    from queue import Queue, Empty  # python 3.x

ON_POSIX = 'posix' in sys.builtin_module_names

def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

pianobar = Popen(['pianobar'], stdin=PIPE, stdout=PIPE, bufsize=1, close_fds=ON_POSIX)
q = Queue()
t = Thread(target=enqueue_output, args=(pianobar.stdout, q))
t.daemon = True # thread dies with the program
t.start()

# ... do other things here

# read line without blocking
#try:  line = q.get_nowait() # or q.get(timeout=.1)
#except Empty:
#    print('no output yet')
#else: # got line
#    # ... do something with line

In [35]:
try:  line = q.get_nowait() # or q.get(timeout=.1)
except Empty:
    print('Reached the end of current output')
else:
    print(line)

b'Welcome to pianobar (2013.09.15)! Press ? for a list of commands.\n'


In [4]:
test = r'	10) q   Cream Radio'
p = re.compile('(\d{1,})\)[\sqS]{5}(.*)')
m = p.search(test)
(i, n) = m.groups()
print(i)
print(n)

10
Cream Radio


In [36]:
channels = []
while True:
    try:  line = q.get_nowait() # or q.get(timeout=.1)
    except Empty:
        print('Reached the end of current output')
        break
    else: # got line
        test = line.rstrip().decode("utf-8")
        m = p.search(test)
        print(test)
        if m is not None:
            (index, name) = p.search(test).groups()
            print("found: {} {}".format(name, index))
            channels.append(name)
            
print(channels)

(i) Login... Ok.
(i) Get stations... Ok.
	 0) q   Abba Radio
	 1) q   Adele Radio
	 2) q   Aerosmith Radio
	 3) q   Alternative Pop/Rock
	 4) q   Billy Joel Radio
	 5) q   Bond Radio
	 6) q   Cake Radio
	 7) q   Carry On Wayward Son Radio
	 8) q   Classic Rock
	 9) q   Classical Guitar Radio
	10) q   Cream Radio
	11) q S Disney (Children's) Radio
	12) q S Easy Listening
	13) q   Faunts Radio
	14) q   Firework Radio
	15) q   Flogging Molly Radio
	16) q S Folk / Country Rock
	17) q   Frou Frou Radio
	18) q   Galileo Radio
	19) q   George Frideric Handel Radio
	20) q   Grazing In The Grass Radio
	21) q   Instrumental Folk
	22) q   Irish/Celtic/Folk
	23) q   James Blunt Radio
	24) q   Johann Sebastian Bach Radio
	25) q   Linkin Park Radio
	26)   S Metallica Live
	27) q   Modest Mouse Radio
	28) q   Muse Radio
	29) q   Norah Jones Radio
	30) q   Of Monsters & Men Radio
	31)   S Pandora Premieres
	32) q   Pat Benatar Radio
	33) q   Pop / Rock
	34)  Q  QuickMix
	35) q   R.E.M. Radio
	36) q   

In [21]:
channel_index = 0
level = 'menu'

In [22]:
def start_channel(channel_index):
    cmd = bytearray(repr(channel_index)+'\n', 'utf-8')
    print(cmd)
    pianobar.stdin.write(cmd)
    pianobar.stdin.flush()

In [48]:
def next():
    cmd = bytearray('n', 'utf-8')
    print(cmd)
    pianobar.stdin.write(cmd)
    pianobar.stdin.flush()

In [45]:
start_channel(0)

bytearray(b'0\n')


In [12]:
nch = 5
select_channel = 4
print(select_channel)
for ii in range(15):
    select_channel = (select_channel-1) % nch
    print(select_channel)

4
3
2
1
0
4
3
2
1
0
4
3
2
1
0
4


In [27]:
line = q.get_nowait()

In [28]:
line

b'\x1b[2K|>  "(I Can\'t Get No) Satisfaction" by "The Rolling Stones" on "Big Hits (High Tide And Green Grass)"\n'

In [28]:
line = q.get_nowait()
print(line)

b'\x1b[2K|>  "Ramble On (Rough Mix With Vocal)" by "Led Zeppelin" on "Led Zeppelin II (Deluxe Edition)"\n'


In [31]:
p = re.compile('"(.*)" by "(.*)" on "(.*)"')
m = p.search(line.decode('utf-8'))
(song, band, album) = m.groups()
print(song)
print(band)
print(album)

Ramble On (Rough Mix With Vocal)
Led Zeppelin
Led Zeppelin II (Deluxe Edition)


In [41]:
now_playing = ''
def update_now_playing():
    p = re.compile('"(.*)" by "(.*)" on "(.*)"')
    while True:
        try:  line = q.get_nowait() # or q.get(timeout=.1)
        except Empty:
            print('Finished Searching for song.')
            break
        else: # got line
            test = line.rstrip().decode("utf-8")
            print(test)
            m = p.search(test)
            #print(test)
            if m is not None:
                (song, band, album) = p.search(test).groups()
                print("found: {} by {} on {}".format(song, band, album))
                now_playing = song
                return True
    return False


In [49]:
next()

bytearray(b'n')


In [50]:
update_now_playing()

|>  "You're So Vain" by "Carly Simon" on "No Secrets"
found: You're So Vain by Carly Simon on No Secrets


True

In [12]:
now_playing

''

In [16]:
line

b'\x1b[2K\t49) q   What The Cowgirls Do Radio\n'

In [51]:
pianobar.terminate()